## Auto mode (single uav)

This is for testing propouses

In [ ]:
# Custom modules
from config import Color
from helpers import clean  #, local2global
from mavlink.customtypes.location import ENUPose, GRAPose
from plan import Plan
from simulator import (
    QGC,
    ConfigGazebo,
    ConfigNovis,
    ConfigQGC,
    Gazebo,
    NoVisualizer,
    Simulator,
)

clean()

## Simulation Configuration

In [ ]:
gra_origin = GRAPose(lat=-35.3633280, lon=149.1652241,alt=0,heading=90)   # east, north, up, heading
enu_origin = ENUPose(x=0, y=0, z=gra_origin.alt, heading=gra_origin.heading) 

base_home = ENUPose(5,5,0,45)
base_path = Plan.create_square_path(side_len=10, alt=5,heading=0)

color =Color.BLUE

## Gazebo Congiguration

In [ ]:
gaz_config = ConfigGazebo(origin= enu_origin,
                          world_path="simulator/gazebo/worlds/runway.world")

gaz_config.add(base_path=base_path,base_home=base_home,color=color)
gaz_config.show()

## QGroundControl Configuration

In [ ]:
qgc_config = ConfigQGC(origin= gra_origin)

qgc_config.add(base_path=base_path,base_home=base_home,color=color)
qgc_config.show()

## No Visualizer

In [ ]:
novis_config = ConfigNovis(origin=gra_origin)
novis_config.add(base_home=base_home)

In [ ]:
novis=NoVisualizer(novis_config)

gaz= Gazebo(gaz_config,gra_origin)

qgc=QGC(qgc_config)

## Launch Simulatotion

In [ ]:
simulator = Simulator(
	visualizers=[novis],
    gcs_names=['simple GCS'],
	gcs_sysids=[[1]],
	missions=[veh.mission for veh in qgc_config.vehicles],
	terminals=['gcs'],
	verbose=2,
)
orac = simulator.launch()

## Oracle checking

In [ ]:
orac.run()